In [1]:
import numpy as np
from scipy.special import comb
from scipy.stats import ttest_rel
import cv2
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from math import floor
import json

In [2]:
plt.rcParams['font.sans-serif'] = 'Times New Roman'

In [3]:
dires = [(-1, 0), (-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1)]
dires = [np.array(x) for x in dires]

def dfs_length(mp, p):
    mp[p[0]][p[1]] = 255
    
    length = 0
    
    for dire in dires:
        new_p = p + dire
        if new_p[0] >= 0 and new_p[0] < mp.shape[0] and new_p[1] >= 0 and new_p[1] < mp.shape[1] and mp[new_p[0]][new_p[1]] == 0:
            length += np.linalg.norm(dire)
            length += dfs_length(mp, new_p)
    
    return length

def calc_length(mp, sig):
    mp = mp.copy()
    length = 0
    for i in range(sig.shape[0]):
        if mp[sig[i][0]][sig[i][1]] == 0:
            length += dfs_length(mp, (sig[i][0], sig[i][1]))
    return length

def calc_area(mp):
    cnt_y, cnt_x = np.nonzero(255 - im)
    return len(cnt_y)

In [4]:
rct_peri = []
rct_a = []

ang = [[],[]]
length = [[],[]]

for person in range(1, 8):
    print(f'{round((person - 1) / 4 * 100)}%', person)
    for pic in range(1, 21):
        im = cv2.imread(f'./skel/{person}/{pic}.png', cv2.IMREAD_GRAYSCALE)

        typ = (pic - 1) // 10
        cnt_y, cnt_x = np.nonzero(255 - im)
        cnt = np.dstack((cnt_y, cnt_x)).squeeze()
        rect = cv2.minAreaRect(np.array(cnt))
        box = cv2.boxPoints(rect)

        box_y = box[:, 0]
        box_x = box[:, 1]

        d0 = np.linalg.norm(box[0] - box[1])
        d1 = np.linalg.norm(box[1] - box[2])

        if d0 < d1:
            slope = -(box[1][0] - box[2][0]) / (box[1][1] - box[2][1])
        else:
            slope = -(box[1][0] - box[0][0]) / (box[1][1] - box[0][1])
        plt.show()
        ang[typ].append(np.arctan(slope)*180/np.pi)
        rct_peri.append(2 * (d0 + d1))
        rct_a.append(d0 * d1)

        length[typ].append(calc_length(im, cnt) / rct_peri[-1])

0% 1
25% 2
50% 3
75% 4
100% 5
125% 6
150% 7


In [5]:
pvalue = [[]]
for i in range(1):
    pvalue[i].append(ttest_rel(ang[0],ang[i+1]))
pvalue

[[Ttest_relResult(statistic=1.0671313569779093, pvalue=0.289632073038585)]]

In [6]:
pvalue = [[]]
for i in range(1):
    pvalue[i].append(ttest_rel(length[0],length[i+1]))
pvalue

[[Ttest_relResult(statistic=-1.4310193106986935, pvalue=0.15693831900287142)]]

In [8]:
dists = [[],[]]

for person in range(1, 8):
    print(f'{round((person - 1) / 4 * 100)}%', person)
    for pic in range(1, 21):
        im = cv2.imread(f'./skel/{person}/{pic}.png', cv2.IMREAD_GRAYSCALE)

        col = []
        for j in range(im.shape[1]):
            pix = (1 - (im[:, j] // 255)).sum()
            col.append(pix)

        typ = (pic - 1) // 10
        if pic % 10 > 5:
            supposed = 3
        else:
            supposed = 2

        splits = []
        start = 0
        flag = False
        for i in range(im.shape[1]):
            if col[i] > 0 and not flag:
                start = i
                flag = True
            elif col[i] == 0 and flag:
                splits.append((start, i))
                flag = False     

        actual = len(splits)
        for i in range(0, actual - supposed):
            cnt = []
            for i, sp in enumerate(splits):
                cnt.append((np.sum(col[sp[0]:sp[1]]), i))
            cnt.sort(key = lambda x: x[0])

            idx = []
            pos = cnt[0][1]
            idx.append(pos)


            dis1 = 224
            dis2 = 224
            if pos - 1 >= 0:
                dis1 = splits[pos][0] - splits[pos - 1][1]
            if pos + 1 < len(splits):
                dis2 = splits[pos + 1][0] - splits[pos][1]
            if dis1 > dis2:
                splits[pos + 1] = (splits[pos][0], splits[pos + 1][1])
            else:
                splits[pos - 1] = (splits[pos - 1][0], splits[pos][1])

            for i in reversed(idx):
                splits.pop(i)

        acutal = len(splits)
        dist = []
        for _ in range(0, supposed - actual): dist.append(0)
        for i in range(len(splits) - 1):
            dist.append(splits[i + 1][0] - splits[i][1])
        dists[typ].append(np.array(dist).mean()/rct_peri[(person - 1) * 20 + pic - 1])

0% 1
25% 2
50% 3
75% 4
100% 5
125% 6
150% 7


In [9]:
pvalue = [[],[]]
for i in range(1):
    pvalue[i].append(ttest_rel(dists[0],dists[i+1]))
pvalue

[[Ttest_relResult(statistic=1.5615000776974735, pvalue=0.12298098066107567)],
 []]

In [10]:
# Save data
with open('angle.dat', 'w') as f:
    json.dump(ang, f)
with open('length.dat', 'w') as f:
    json.dump(length, f)
with open('dist.dat', 'w') as f:
    json.dump(dists, f)